In [72]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.metrics import *

In [3]:
df = pd.read_csv("laptop_data.csv")
df.drop("Unnamed: 0",axis=1,inplace=True)
df.head()

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832
1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232
2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000
3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360
4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080


In [4]:
df.isna().sum()

Company             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price               0
dtype: int64

In [5]:
df.isna().sum().sum()

0

In [6]:
df.duplicated().sum()

29

In [7]:
df_1 = df.drop_duplicates()
df_1.head()

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832
1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232
2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000
3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360
4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080


In [8]:
print("Company :",df_1["Company"].unique())
print()
print("Company :",df_1["TypeName"].unique())
print()
print("Company :",(df_1["OpSys"].unique()))

Company : ['Apple' 'HP' 'Acer' 'Asus' 'Dell' 'Lenovo' 'Chuwi' 'MSI' 'Microsoft'
 'Toshiba' 'Huawei' 'Xiaomi' 'Vero' 'Razer' 'Mediacom' 'Samsung' 'Google'
 'Fujitsu' 'LG']

Company : ['Ultrabook' 'Notebook' 'Netbook' 'Gaming' '2 in 1 Convertible'
 'Workstation']

Company : ['macOS' 'No OS' 'Windows 10' 'Mac OS X' 'Linux' 'Android' 'Windows 10 S'
 'Chrome OS' 'Windows 7']


In [46]:

df_2 = df_1.copy()

df_2["ScreenResolution"] = list(map(lambda x: re.search("\d\d\d+x\d\d\d+",x).group(), df_1["ScreenResolution"]))
df_2["ScreenResolution"] = list(map(lambda x: tuple(x.split("x")),df_2["ScreenResolution"]))

df_2["Cpu"] = list(map(lambda x: x.split(" ")[-1].lower(),df_1["Cpu"]))
df_2["Cpu"] = list(map(lambda x: re.sub("ghz","",x),df_2["Cpu"]))
df_2["Cpu"] = df_2["Cpu"].astype("float64")

df_2["Ram"] = list(map(lambda x: re.sub("gb","",x.lower()),df_1["Ram"]))
df_2["Ram"] = df_2["Ram"].astype("int64")

df_2["Memory"] = list(map(lambda x: x.split("+"),df_1["Memory"]))
df_2["Memory"] = list(map(lambda x: [i.strip().lower() for i in x],df_2["Memory"]))


df_2["ssd"] = np.nan
df_2["hdd"] = np.nan
df_2["flash storage"] = np.nan

df_2.reset_index(drop=True,inplace=True)


for i in df_2.index.values:
    for j in df_2.loc[i,"Memory"]:
        p = j.split(" ")
        if len(p) > 2:
            p[1] = p[1] + " " + p[2]
            p.pop()
        df_2.loc[i,p[1]] = int(re.search("\d+",p[0]).group())
        h = re.search("\D+",p[0]).group()
        if h=="tb":
            df_2.loc[i,p[1]] = 1024 * df_2.loc[i,p[1]]
        # print(df_2.loc[i,p[1]],end=" ")
   #  print()

df_2.drop("Memory",axis=1,inplace=True)
df_2["ssd"] = df_2["ssd"].fillna(0)
df_2["hdd"] = df_2["hdd"].fillna(0)
df_2["flash storage"] = df_2["flash storage"].fillna(0)
df_2["hybrid"] = df_2["hybrid"].fillna(0)


df_2["Gpu"] = list(map(lambda x: x.split(" ")[0],df_1["Gpu"]))
df_2["Weight"] = list(map(lambda x: float(x[0:len(x)-2].strip()),df_1["Weight"]))


df_2["Screen Length"] = list(map(lambda x:x[0],df_2["ScreenResolution"]))
df_2["Screen Width"] = list(map(lambda x:x[1],df_2["ScreenResolution"]))


df_2.drop("ScreenResolution",axis=1,inplace=True)

df_2.head()


df_2["OpSys"][df_2["OpSys"]=="Mac OS X"] = "macOS"
df_2["OpSys"][df_2["OpSys"]=="Windows 10 S"] = "Windows 10"


df_2["Screen Length"] = df_2["Screen Length"].astype("int64")
df_2["Screen Width"] = df_2["Screen Width"].astype("int64")
df_2["ssd"] = df_2["ssd"].astype("int64")
df_2["hdd"] = df_2["hdd"].astype("int64")
df_2["flash storage"] = df_2["flash storage"].astype("int64")
df_2["hybrid"] = df_2["hybrid"].astype("int64")

df_2.head()

,Company,TypeName,Inches,Cpu,Ram,Gpu,OpSys,Weight,Price,ssd,hdd,flash storage,hybrid,Screen Length,Screen Width
0,Apple,Ultrabook,13.3,2.3,8,Intel,macOS,1.37,71378.6832,128,0,0,0,2560,1600
1,Apple,Ultrabook,13.3,1.8,8,Intel,macOS,1.34,47895.5232,0,0,128,0,1440,900
2,HP,Notebook,15.6,2.5,8,Intel,No OS,1.86,30636.0000,256,0,0,0,1920,1080
3,Apple,Ultrabook,15.4,2.7,16,AMD,macOS,1.83,135195.3360,512,0,0,0,2880,1800
4,Apple,Ultrabook,13.3,3.1,8,Intel,macOS,1.37,96095.8080,256,0,0,0,2560,1600


In [58]:
df_3 = df_2.copy()

le = LabelEncoder()

df_3["Company"] = le.fit_transform(df_2["Company"])
df_3["TypeName"] = le.fit_transform(df_2["TypeName"])

df_3["Gpu"] = le.fit_transform(df_2["Gpu"])
df_3["OpSys"] = le.fit_transform(df_2["OpSys"])

df_3["Price"] = df_3["Price"].round(2)

df_3.head()

,Company,TypeName,Inches,Cpu,Ram,Gpu,OpSys,Weight,Price,ssd,hdd,flash storage,hybrid,Screen Length,Screen Width
0,1,4,13.3,2.3,8,2,6,1.37,71378.68,128,0,0,0,2560,1600
1,1,4,13.3,1.8,8,2,6,1.34,47895.52,0,0,128,0,1440,900
2,7,3,15.6,2.5,8,2,3,1.86,30636.00,256,0,0,0,1920,1080
3,1,4,15.4,2.7,16,0,6,1.83,135195.34,512,0,0,0,2880,1800
4,1,4,13.3,3.1,8,2,6,1.37,96095.81,256,0,0,0,2560,1600


In [65]:
df_3.drop_duplicates(inplace=True)
df_3.reset_index(drop=True,inplace=True)

In [66]:
X = df_3.drop("Price",axis=1)
y = df_3["Price"]

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=np.random.randint(0,1000))

In [71]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [79]:
y_pred = lr.predict(X_test).round(2)

np.round(mean_absolute_error(y_test,y_pred),2)

14618.59

In [82]:
y_pred[0]

78382.27